## Deliverable 2. Create a Customer Travel Destinations Map.

In [87]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [44]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.rename(columns={'Weather Desciption':'Weather Description'}, inplace=True)
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,% Humidity,% Cloudiness,Wind Speed,Weather Description
0,0,Hambantota,LK,6.1241,81.1185,78.49,84,70,1.50,broken clouds
1,1,Lingyuan,CN,41.2400,119.4011,43.57,73,100,10.51,overcast clouds
2,2,Hasaki,JP,35.7333,140.8333,52.20,60,99,6.26,overcast clouds
3,3,Albany,US,42.6001,-73.9662,58.68,91,100,4.00,overcast clouds
4,4,Carnarvon,AU,-24.8667,113.6333,66.27,68,0,16.11,clear sky


In [45]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = int(input("Enter the minimum temperature you'd like to see:"))
max_temp = int(input("Enter the maximum temperature you'd like to see:"))

Enter the minimum temperature you'd like to see:75
Enter the maximum temperature you'd like to see:90


In [46]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
filtered_data = city_data_df.loc[(city_data_df['Max Temp'] > min_temp) & (city_data_df['Max Temp'] < max_temp)]

In [47]:
# 4a. Determine if there are any empty rows.
filtered_data.count()

City_ID                199
City                   199
Country                196
Lat                    199
Lng                    199
Max Temp               199
% Humidity             199
% Cloudiness           199
Wind Speed             199
Weather Description    199
dtype: int64

In [76]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = filtered_data.dropna()

City_ID                196
City                   196
Country                196
Lat                    196
Lng                    196
Max Temp               196
% Humidity             196
% Cloudiness           196
Wind Speed             196
Weather Description    196
dtype: int64

In [77]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,Hambantota,LK,78.49,broken clouds,6.1241,81.1185,
9,Abu Dhabi,AE,84.22,clear sky,24.4667,54.3667,
14,Lorengau,PG,81.73,broken clouds,-2.0226,147.2712,
17,Georgetown,MY,77.90,few clouds,5.4112,100.3354,
18,Alta Floresta,BR,89.42,few clouds,-9.8756,-56.0861,
19,Butaritari,KI,82.27,scattered clouds,3.0707,172.7902,
21,Adrar,MR,84.56,overcast clouds,20.5022,-10.0711,
23,Punto Fijo,VE,82.51,overcast clouds,11.6956,-70.1996,
24,Atuona,PF,77.76,clear sky,-9.8000,-139.0333,
26,Kapaa,US,80.58,clear sky,22.0752,-159.3190,


In [78]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f'{lat},{lng}'
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    data = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = data["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        pass

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [102]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df = hotel_df.dropna()
hotel_df.count()

City                   184
Country                184
Max Temp               184
Weather Description    184
Lat                    184
Lng                    184
Hotel Name             184
dtype: int64

In [103]:
# 8a. Create the output File (CSV)
output_data_file = "./WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [108]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Weather Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [109]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))